<a href="https://colab.research.google.com/github/AnthonyBadmus/Promotional-Campaign-Analysis/blob/main/new_loan_validator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# @title Please enter the URL and Sheet Name
url = 'https://docs.google.com/spreadsheets/d/1WXBj3Oh43If6ppx4YqDLsD-Zu4Yju1Wd/edit#gid=913337563'  # @param {type:"string"}
sheet_name = 'new_loans_data' # @param {type:"string"}
partner = 'Stanbic IBTC' # @param {type:"string"}




In [2]:
!pip install pydantic[email]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 9.9 MB/s eta 0:00:00


# New Loans

### Get Loans Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
from google.colab import auth
from google.auth import default
from google.colab import auth
auth.authenticate_user()
import gspread
from pathlib import Path

creds, _ = default()
gc = gspread.authorize(creds)


def extract_file_id_from_url(url):
    parts = url.split('/')
    file_id_index = parts.index('d') + 1
    file_id = parts[file_id_index]
    return file_id




def get_df(file_id, sheet_name):
    x = Path(f'/content/drive/.shortcut-targets-by-id/{file_id}').iterdir()
    file_path = list(x)[0]
    if file_path.suffix in ('.xlsx','.xls','.xlsb'):
        df = pd.read_excel(file_path,dtype="string")
    elif file_path.suffix == '.csv':
        df = pd.read_csv(file_path, dtype="string")
    elif file_path.suffix == '.gsheet':
        df = pd.DataFrame(gc.open_by_key(file_id).worksheet(sheet_name).get_all_records()).astype("string")
    return df


df = get_df(extract_file_id_from_url(url), sheet_name)



# new_loans_data = df.astype(str).replace('', None).replace("^#[A-Z/]+!?$", None, regex=True)
new_loans_data = df.replace("", None).replace("^#[A-Z/]+!?$", None, regex=True)


date_cols = ['enter_date','disbursement_date','due_date']

def convert_date(df,cols):
  for column in date_cols:
      df[column] =  pd.to_datetime(df[column],
                                  #  format='%d/%m/%Y',
                                   errors='coerce').replace({pd.NaT:None})
  return df


new_loans_data = convert_date(new_loans_data,date_cols)
new_loans_data.head(10)



,user_id,loan_id,loan_instalment_id,enter_date,disbursement_date,due_date,application_name,tags,product_name,uphone,...,account_number,bank_name,original_principal_to_repay,original_interest_to_repay,amount_previously_paid,amount_to_repay_today_incl_late_fees,original_amount_to_repay,channel,assigned_to,company
0,100815024,100815024,0034865234,2024-05-22,2023-09-05,2024-05-07,Olaoluwa Abiodun Hussein,UNSECURED PERSONAL LOAN,EZCSH,8080208041,...,0009820857,Stanbic IBTC,470000,0,16.17,16.17,151705.49,work_flow,BFR00077,Stanbic IBTC
1,101386898,101386898,0037224706,2024-05-22,2023-05-26,2024-05-07,Sunusi Muhammad Lawan,UNSECURED PERSONAL LOAN,EZCSH,8130073777,...,0016824088,Stanbic IBTC,792000,8221.86,27159.21,27159.21,296004.61,work_flow,BFR00077,Stanbic IBTC
2,100586280,100586280,0038807162,2024-05-22,2024-04-27,2024-05-07,John Fidelis Sabo,UNSECURED PERSONAL LOAN,EZCSH,8101336290,...,0007183525,Stanbic IBTC,380000,5447.45,11577.96,11577.96,199018.15,work_flow,BFR00077,Stanbic IBTC
3,101751927,101751927,0040061774,2024-05-22,2023-02-07,2024-05-07,Rotimi Ibrahim Idowu,UNSECURED PERSONAL LOAN,EZCSH,8083643709,...,0019906178,Stanbic IBTC,999000,0,18161,18161,319995.26,work_flow,BFR00077,Stanbic IBTC
4,100528678,100528678,0051457706,2024-05-22,2023-11-28,2024-05-07,Mohammed Umar,UNSECURED PERSONAL LOAN,EZCSH,8075235982,...,0006494844,Stanbic IBTC,350000,8621.81,20873.27,20873.27,296272.22,work_flow,BFR00077,Stanbic IBTC
5,101216068,101216068,0055020317,2024-05-22,2024-02-19,2024-05-07,Mubarak Adamu Nasidi,UNSECURED PERSONAL LOAN,EZCSH,8067109594,...,0014674849,Stanbic IBTC,20000,451.11,3728.87,3728.87,13943.79,work_flow,BFR00077,Stanbic IBTC
6,104221381,104221381,0055596629,2024-05-22,2024-04-04,2024-05-07,Abdulsamod Olayiwola Toriola,UNSECURED PERSONAL LOAN,EZCSH,9017645046,...,0049557760,Stanbic IBTC,110000,0,28110.45,28110.45,28110.45,work_flow,BFR00077,Stanbic IBTC
7,100508140,100508140,0055761571,2024-05-22,2024-03-09,2024-05-07,Racheal Oluwaninyo Fayomi,UNSECURED PERSONAL LOAN,EZCSH,9072332157,...,0006232000,Stanbic IBTC,20000,0,6079.89,6079.89,12887.27,work_flow,BFR00077,Stanbic IBTC
8,103006369,103006369,0055846623,2024-05-22,2024-03-11,2024-05-07,Abdulgafar Mahmud Ikharo,UNSECURED PERSONAL LOAN,EZCSH,7016380150,...,0036003733,Stanbic IBTC,350000,0,21088.39,21088.39,328016.82,work_flow,BFR00077,Stanbic IBTC
9,103082106,103082106,0057284977,2024-05-22,2024-04-17,2024-05-07,Zayyanu Salihu,UNSECURED PERSONAL LOAN,EZCSH,8141755440,...,0035223293,Stanbic IBTC,160000,0,8.49,8.49,153545.71,work_flow,BFR00077,Stanbic IBTC


### Validate Headers

In [8]:


def validate_headers(df):
    headers = [

          'user_id','loan_id','loan_instalment_id','disbursement_date','enter_date',
          'due_date','application_name','tags','email','uphone','bvn_phone','bvn','gender','dob','product_name',
            'original_principal_to_repay','original_amount_to_repay','original_interest_to_repay','original_amount_to_repay',
          'original_late_fees','amount_to_repay_today_incl_late_fees', 'customer_discount','days_late','amount_previously_paid',
            'assigned_to','account_number','bank_name','channel','company','lga','state'

            ]

    new_loans_data_cols = df.columns
    check_headers = []
    for column in new_loans_data_cols:
      if column not in headers:
          check_headers.append(column)
    return check_headers


checking = validate_headers(new_loans_data)

if checking:
    print(f'Incorrect/Missing header(s): {checking}')
else:
    print('Headers are correct.')






Headers are correct.


### New Loans Validator

In [ ]:
from pydantic import BaseModel, PositiveFloat, PositiveInt,EmailStr, constr, validator,Field,field_validator, ConfigDict
import pandas as pd
from pydantic.types import UUID4
from datetime import date
from typing import Union, Optional, Literal


class New_loans(BaseModel):
    model_config = ConfigDict(extra="forbid")

    # user_id: str = Field(..., pattern=r'^[A-Za-z0-9]+$')
    # loan_id: str = Field(..., pattern=r'^[A-Za-z0-9]+$')
    user_id: str | UUID4 | int
    loan_id: str | UUID4 | int
    loan_instalment_id: Optional[str] |int | None = None
    disbursement_date: date = Field(..., le=date.today())
    enter_date: date = Field(..., le=date.today())
    due_date: date
    application_name: str = Field(..., pattern=r"^[a-zA-Z\s']+$")
    # tags: Union[int, str]
    tags:  Optional[str] |int | None = None
    product_name:  Optional[str] | None = None
    email: Optional[EmailStr] | None = None
    uphone: int = constr(min_length=10, max_length=10)
    original_principal_to_repay: float|int = Field(0, ge=0)
    original_interest_to_repay: float|int = Field(0, ge=0)
    original_amount_to_repay: float|int = Field(0, ge=0)
    amount_to_repay_today_incl_late_fees: float|int = Field(0, ge=0)
    days_late: Optional[int] | None = None
    # amount_previously_paid: Optional[float|int]  = Field(0, ge=0)
    amount_previously_paid: Optional[float|int] | None = None
    assigned_to: str
    account_number: str = Field(..., min_length=10, max_length=10)
    bank_name: str = Field(..., pattern=r"^[a-zA-Z\s']+$")
    channel: str = Field(..., pattern=r"^[a-zA-Z\s'_]+$")
    company: str = Field(..., pattern=r"^[a-zA-Z\s']+$")
    state: Optional[str] | None = None
    bvn_phone: Optional[str] | None = Field(None, max_length=10, min_length=10)
    customer_discount: Optional[float] = Field(0, ge=0)
    original_late_fees: Optional[float] = Field(0, ge=0)
    gender: Optional[Literal['Male','Female','Other']] = 'Other'
    dob: Optional[date] | None = None
    bvn: Optional[str] = Field(None, min_length=11, max_length=11)


    @field_validator("uphone")
    def validate_uphone(cls, value):
        allowed_prefixes = ["80", "70", "90","81","91"]
        first_two_digits = str(value)[:2]

        if first_two_digits not in allowed_prefixes:
            raise ValueError(f"Invalid uphone number. Must be one of {allowed_prefixes}")

        return value


    def validate_bvn_phone(cls, value):
        if value is None or value == "":
            return value

        allowed_prefixes = ["80", "70", "90", "81", "91"]
        first_two_digits = str(value)[:2]

        if first_two_digits not in allowed_prefixes:
            raise ValueError(f"Invalid bvn_phone number. Must be one of {allowed_prefixes}")

        return value



nl_data = new_loans_data.to_dict(orient='records')

validation_results = []

for index, record in enumerate(nl_data, start=2):
    validation_result = {"row_number": index,"loan_id":record['loan_id'], "validation_status": ""}
    try:
        new_loans = New_loans(**record)
        validation_result["validation_status"] = "Validation successful"

    except ValueError as e:
            e = ",\n".join([f"{err['loc'][0]}: {err['msg']}" for err in e.errors() ])
            validation_result["validation_status"] = f"Validation failed: {e}"

    validation_results.append(validation_result)


failed_validations = [result for result in validation_results if "Validation failed" in result["validation_status"]]


nl_failed_validations = pd.DataFrame(failed_validations)
count_nl_failed_validations = len(nl_failed_validations)
print(f'There are {count_nl_failed_validations} failed validations')
nl_failed_validations
# nl_failed_validations.iloc[0][2]


There are 0 failed validations


""


### New Loans Validation Logs

In [ ]:
from google.colab import drive
drive.mount("/ShardDrives/")

Mounted at /ShardDrives/


In [ ]:
partner = partner
date = date.today()

shared_drive_path = '/ShardDrives/Shareddrives/ops-techsupport-data/logs/'
file_name = f'{partner}_new_loans_failed_validations_{date}.csv'

nl_failed_validations.to_csv(shared_drive_path + file_name, index=False)